# How to use XLEMOO: Explainable Learnable Multiobjective Optimization

XLEMOO is a hybrid evolutionary-ML approach that combines evolutionary algorithms
(Darwinian mode) with interpretable machine learning (Learning mode) to find
near-Pareto optimal solutions while providing **explainability** about what makes
solutions good.

The core idea is that by training interpretable ML models on the population history,
we can extract human-readable rules describing what characterizes high-performing
solutions. These rules can then be used to instantiate new candidate solutions
directly.

**Reference:**
Misitano, G. (2024). Towards Explainable Multiobjective Optimization: XLEMOO.
*ACM Trans. Evol. Learn. Optim.*, 4(1). https://doi.org/10.1145/3626104

**Prerequisites:** Before proceeding, make sure you are familiar with:

- [How evolutionary algorithms are structured in DESDEO](../../explanation/templates_and_pub_sub)
- [How to configure evolutionary algorithms with the Pydantic interface](../ea_options)
- [How multiobjective optimization problems are defined](../../explanation/problem_format.ipynb)

## How XLEMOO works

XLEMOO alternates between two modes:

**Darwinian mode** uses a standard evolutionary algorithm (e.g., RVEA or NSGA-III)
with crossover, mutation, and selection operators. This is identical to running
a conventional EA.

**Learning mode** replaces the standard evolutionary operators with an ML-based
pipeline:

1. Read population history from the archive.
2. Rank solutions by a scalar fitness indicator and split into a
   **H-group** (high-performing) and **L-group** (low-performing).
3. Train an interpretable ML classifier to distinguish H from L.
4. Extract rules from the classifier and instantiate new candidate
   solutions based on those rules.
5. Evaluate the candidates using the shared evaluator.
6. Select the best solutions using the same Darwinian selector.

In DESDEO, these responsibilities are split across the instantiator operator,
the shared evaluator, and the Darwinian selector:

```
Darwinian mode (N iterations):
  offspring       = crossover(population)
  offspring       = mutate(offspring)
  offspring_eval  = evaluator.evaluate(offspring)
  population      = selection(parents, offspring)

Learning mode (M iterations):
  candidates      = instantiator(population)        # ML rules -> variables
  candidates_eval = evaluator.evaluate(candidates)   # shared evaluator
  population      = selection(parents, candidates)   # same selector
```

Because the evaluation step goes through the shared `EMOEvaluator`,
all components (terminator, archives, etc.) are correctly notified about
the function evaluations performed during learning mode. Using the same
selector (e.g., RVEA) for both modes preserves population diversity
across the Darwinian-Learning mode boundary.

## Running XLEMOO with ASF-guided learning

The simplest way to run XLEMOO is with the pre-configured
`xlemoo_nsga2_options()`, which pairs NSGA-II as the Darwinian selector
with a Decision Tree classifier for learning mode.

In this example we use the **Achievement Scalarizing Function (ASF)** as
the fitness indicator for the H/L group ranking. ASF incorporates a
reference point provided by the decision maker, guiding the learning mode
to focus on solutions near that preferred region. The reference point
$z = (0.5, 0.5, 0.5)$ sits near the center of the DTLZ2 Pareto front
(the unit sphere in the positive octant).

In [ ]:
import warnings

warnings.filterwarnings("ignore")

import numpy as np

from desdeo.emo.options.algorithms import emo_constructor, xlemoo_nsga2_options
from desdeo.emo.options.xlemoo_selection import XLEMOOSelectorOptions
from desdeo.problem.testproblems import dtlz2

# Define a 3-objective, 6-variable DTLZ2 problem
problem = dtlz2(n_objectives=3, n_variables=6)

# Get pre-configured XLEMOO-NSGA2 options
options = xlemoo_nsga2_options()

# Use ASF with a reference point for H/L group ranking
reference_point = [0.5, 0.5, 0.5]
options.template.learning_selection = XLEMOOSelectorOptions(
    fitness_indicator="asf",
    asf_reference_point=reference_point,
    asf_weights=[1.0, 1.0, 1.0],
)

# Build and run the solver
solver, extras = emo_constructor(emo_options=options, problem=problem)
results = solver()

print(f"Solutions found: {len(results.optimal_outputs)}")
print(results.optimal_outputs.head())

Let's visualize the Pareto front approximation. For DTLZ2 with 3 objectives,
the true Pareto front lies on the unit sphere.

In [ ]:
import plotly.express as px

fig = px.scatter_3d(
    results.optimal_outputs,
    x="f_1",
    y="f_2",
    z="f_3",
    title="XLEMOO-NSGA2 on DTLZ2 (3 objectives)",
)
fig.update_traces(marker=dict(size=3))
fig.show(renderer="notebook", include_plotlyjs="cdn")

In [ ]:
# Verify proximity to unit sphere
obj_values = results.optimal_outputs[["f_1", "f_2", "f_3"]].to_numpy()
norms = np.sqrt(np.sum(obj_values**2, axis=1))
print(f"Median distance to unit sphere: {np.median(norms):.4f}")
print(f"Min: {norms.min():.4f}, Max: {norms.max():.4f}")

## Understanding the default configuration

Let's inspect the options returned by `xlemoo_nsga2_options()` to understand
the default configuration.

In [ ]:
from rich.pretty import pprint

options = xlemoo_nsga2_options()
pprint(options)

The key XLEMOO-specific settings live in `options.template.learning_selection`
(`XLEMOOSelectorOptions`), and in the template-level cycle parameters:

| Parameter | Default | Description |
|---|---|---|
| `darwin_iterations_per_cycle` | 10 | EA generations per cycle before switching to learning mode |
| `learning_iterations_per_cycle` | 1 | Learning mode iterations per cycle |
| `ml_model_type` | `DecisionTree` | Interpretable ML classifier to use |
| `h_split` | 0.1 | Fraction of best solutions for the H-group |
| `l_split` | 0.1 | Fraction of worst solutions for the L-group |
| `instantiation_factor` | 2.0 | How many candidates to generate (multiplier of population size) |
| `fitness_indicator` | `naive_sum` | How to compute scalar fitness for ranking (`naive_sum`, `asf`, `hypervolume`) |
| `asf_reference_point` | None | Reference point for ASF (required when `fitness_indicator="asf"`) |
| `asf_weights` | None | Weights for ASF (defaults to equal weights if None) |
| `generation_lookback` | 0 | How many recent generations to use (0 = all history) |
| `ancestral_recall` | 0 | Earliest generations to always include |
| `unique_only` | False | Deduplicate solutions before training |

Using `asf` with a decision-maker-supplied reference point is the recommended
configuration, as it directs the learning mode toward the preferred region of
the Pareto front.

## Customizing the ML model

XLEMOO supports several interpretable ML classifiers. The available types are:

- `DecisionTree` (scikit-learn `DecisionTreeClassifier`)
- `SkopeRules` (from [imodels](https://github.com/csinva/imodels))
- `Slipper` (from imodels)
- `BoostedRules` (from imodels)

You can change the ML model and pass additional keyword arguments to its
constructor.

In [ ]:
problem = dtlz2(n_objectives=3, n_variables=6)

options = xlemoo_nsga2_options()
# Reduce generations for a quicker run
options.template.termination.max_generations = 50

# Switch to SkopeRules with custom parameters and ASF fitness
options.template.learning_selection = XLEMOOSelectorOptions(
    ml_model_type="SkopeRules",
    ml_model_kwargs={
        "precision_min": 0.1,
        "n_estimators": 30,
        "max_features": None,
        "max_depth": None,
    },
    h_split=0.2,
    l_split=0.2,
    instantiation_factor=5.0,
    fitness_indicator="asf",
    asf_reference_point=[0.5, 0.5, 0.5],
    asf_weights=[1.0, 1.0, 1.0],
)

solver, extras = emo_constructor(emo_options=options, problem=problem)
results = solver()

print(f"Solutions found: {len(results.optimal_outputs)}")

## Choosing a fitness indicator

The fitness indicator determines how solutions are ranked into H-group
(good) and L-group (bad) during the learning mode. Three options are
available:

**`asf`** (Achievement Scalarizing Function): Uses a reference point and
weights to rank solutions by proximity to the decision maker's preference.
This is the recommended indicator, as it focuses the ML model on the
region of interest.

$$\text{ASF}(f) = \max_i \bigl[ w_i (f_i - z_i) \bigr] + \rho \sum_i w_i (f_i - z_i)$$

**`naive_sum`**: Sums all minimized target values. Simple and fast,
but does not incorporate preference information.

**`hypervolume`**: Uses individual hypervolume contribution. More expensive
to compute, but provides a distribution-aware ranking.

Below we compare the effect of changing the reference point. A reference
point closer to the origin focuses the H-group on solutions in that corner.

In [ ]:
problem = dtlz2(n_objectives=3, n_variables=6)

# Run with a reference point biased toward f_1
options = xlemoo_nsga2_options()
options.template.termination.max_generations = 50
options.template.learning_selection = XLEMOOSelectorOptions(
    fitness_indicator="asf",
    asf_reference_point=[0.2, 0.8, 0.8],
    asf_weights=[1.0, 1.0, 1.0],
)

solver, extras = emo_constructor(emo_options=options, problem=problem)
results = solver()

obj_values = results.optimal_outputs[["f_1", "f_2", "f_3"]].to_numpy()
print(f"Solutions found: {len(results.optimal_outputs)}")
print("Reference point: [0.2, 0.8, 0.8] (biased toward low f_1)")
print(f"Objective ranges: {obj_values.min(axis=0).round(3)} to {obj_values.max(axis=0).round(3)}")

## Controlling the Darwinian/Learning mode cycle

The `darwin_iterations_per_cycle` and `learning_iterations_per_cycle`
parameters control how many iterations are spent in each mode before switching.

For example, with 100 total generations, 10 Darwinian iterations per cycle,
and 1 learning iteration per cycle, the algorithm will run approximately
9 full cycles (each consisting of 10 EA generations + 1 learning step =
11 generations counted by the terminator).

In [ ]:
problem = dtlz2(n_objectives=3, n_variables=6)

options = xlemoo_nsga2_options()
options.template.termination.max_generations = 60

# Use ASF for H/L ranking
options.template.learning_selection = XLEMOOSelectorOptions(
    fitness_indicator="asf",
    asf_reference_point=[0.5, 0.5, 0.5],
    asf_weights=[1.0, 1.0, 1.0],
)

# More frequent learning: 5 Darwinian + 2 learning per cycle
options.template.darwin_iterations_per_cycle = 5
options.template.learning_iterations_per_cycle = 2

solver, extras = emo_constructor(emo_options=options, problem=problem)
results = solver()

print(f"Solutions found: {len(results.optimal_outputs)}")

## Generation lookback and ancestral recall

By default, the learning mode uses the entire population history to train
the ML model (`generation_lookback=0`). You can limit this to only recent
generations, which may be useful for problems where the landscape changes
or when you want the ML model to focus on recently discovered good solutions.

- `generation_lookback`: Only use the last N generations (0 = use all).
- `ancestral_recall`: Always include the first N generations alongside
  the lookback window. Useful for preserving early exploration diversity.

In [ ]:
problem = dtlz2(n_objectives=3, n_variables=6)

options = xlemoo_nsga2_options()
options.template.termination.max_generations = 60

# Use ASF for H/L ranking
options.template.learning_selection = XLEMOOSelectorOptions(
    fitness_indicator="asf",
    asf_reference_point=[0.5, 0.5, 0.5],
    asf_weights=[1.0, 1.0, 1.0],
)

# Only look at the last 20 generations, but always include the first 3
options.template.learning_selection.generation_lookback = 20
options.template.learning_selection.ancestral_recall = 3

solver, extras = emo_constructor(emo_options=options, problem=problem)
results = solver()

print(f"Solutions found: {len(results.optimal_outputs)}")

## Accessing the learning archive

The `ConstructorExtras` object returned by `emo_constructor` includes a
`learning_archive` that stores the full history of all evaluated solutions
(from both Darwinian and Learning modes). This can be used for post-hoc
analysis.

In [ ]:
problem = dtlz2(n_objectives=3, n_variables=6)

options = xlemoo_nsga2_options()
options.template.termination.max_generations = 50
options.template.learning_selection = XLEMOOSelectorOptions(
    fitness_indicator="asf",
    asf_reference_point=[0.5, 0.5, 0.5],
    asf_weights=[1.0, 1.0, 1.0],
)

solver, extras = emo_constructor(emo_options=options, problem=problem)
results = solver()

# The learning archive contains all evaluated solutions with generation numbers
archive_df = extras.learning_archive.solutions
print(f"Total solutions in learning archive: {len(archive_df)}")
print(f"Generations covered: {archive_df['generation'].min()} to {archive_df['generation'].max()}")
print(f"\nArchive columns: {archive_df.columns}")

## Extracting and interpreting rules

One of the key features of XLEMOO is the ability to extract human-readable
rules from the ML model. The `desdeo.explanations.rule_interpreters` module
provides utilities for this.

The following example trains a Decision Tree on the final archive data and
extracts rules describing what characterizes the best solutions. We use
the same ASF indicator with a reference point to rank solutions into
H-group and L-group.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

from desdeo.explanations.rule_interpreters import find_all_paths

problem = dtlz2(n_objectives=3, n_variables=6)

options = xlemoo_nsga2_options()
options.template.termination.max_generations = 50
options.template.learning_selection = XLEMOOSelectorOptions(
    fitness_indicator="asf",
    asf_reference_point=[0.5, 0.5, 0.5],
    asf_weights=[1.0, 1.0, 1.0],
)

solver, extras = emo_constructor(emo_options=options, problem=problem)
results = solver()

# Extract variable and target data from the archive
archive_df = extras.learning_archive.solutions
var_cols = [f"x_{i}" for i in range(1, 7)]
target_cols = [f"f_{i}_min" for i in range(1, 4)]

X = archive_df[var_cols].to_numpy()
targets = archive_df[target_cols].to_numpy()

# Compute ASF fitness (same as the learning mode uses) and split into H/L groups
z = np.array([0.5, 0.5, 0.5])
w = np.array([1.0, 1.0, 1.0])
rho = 1e-6
weighted = (targets - z) * w
fitness = np.max(weighted, axis=1) + rho * np.sum(weighted, axis=1)

sorted_idx = np.argsort(fitness)
n = len(sorted_idx)
h_count = max(1, int(0.1 * n))
l_count = max(1, int(0.1 * n))

h_idx = sorted_idx[:h_count]
l_idx = sorted_idx[-l_count:]

# Train a Decision Tree
X_train = np.vstack((X[h_idx], X[l_idx]))
y_train = np.hstack(
    (
        np.ones(h_count, dtype=int),
        -np.ones(l_count, dtype=int),
    )
)

tree = DecisionTreeClassifier(max_depth=4, random_state=42)
tree.fit(X_train, y_train)

# Extract paths from the tree
paths = find_all_paths(tree)

# Print rules for paths classified as "good" (class 1)
print(f"Total paths in tree: {len(paths)}")
print(f"Paths classified as H-group (good): {sum(1 for p in paths if p['classification'] == 1)}")
print()

for i, path in enumerate(paths):
    if path["classification"] == 1:
        print(f"--- Path {i} (samples: {path['samples']}, impurity: {path['impurity']:.3f}) ---")
        for rule in path["rules"]:
            feature_idx, op, threshold = rule[0], rule[1], rule[2]
            var_name = f"x_{feature_idx + 1}"
            op_str = "<=" if op == "lte" else ">"
            print(f"  {var_name} {op_str} {float(threshold):.4f}")
        print()

The rules above describe decision variable conditions that characterize
high-performing solutions. For DTLZ2, the true Pareto-optimal solutions have
$x_i = 0.5$ for $i \geq 3$ (the distance variables), so we expect the rules
to reflect conditions near $x_i \approx 0.5$ for those variables.

## Comparing XLEMOO with standard NSGA-II

Let's compare the performance of XLEMOO-NSGA2 against standard NSGA-II on the
same problem and budget.

In [ ]:
from desdeo.emo.options.algorithms import nsga2_options

problem = dtlz2(n_objectives=3, n_variables=6)
max_gens = 100

# Standard NSGA-II
nsga2_opts = nsga2_options()
nsga2_opts.template.termination.max_generations = max_gens
nsga2_solver, _ = emo_constructor(emo_options=nsga2_opts, problem=problem)
nsga2_results = nsga2_solver()

# XLEMOO-NSGA2 with ASF
xlemoo_opts = xlemoo_nsga2_options()
xlemoo_opts.template.termination.max_generations = max_gens
xlemoo_opts.template.learning_selection = XLEMOOSelectorOptions(
    fitness_indicator="asf",
    asf_reference_point=[0.5, 0.5, 0.5],
    asf_weights=[1.0, 1.0, 1.0],
)
xlemoo_solver, _ = emo_constructor(emo_options=xlemoo_opts, problem=problem)
xlemoo_results = xlemoo_solver()

# Compare distance to unit sphere
nsga2_obj = nsga2_results.optimal_outputs[["f_1", "f_2", "f_3"]].to_numpy()
xlemoo_obj = xlemoo_results.optimal_outputs[["f_1", "f_2", "f_3"]].to_numpy()

nsga2_norms = np.sqrt(np.sum(nsga2_obj**2, axis=1))
xlemoo_norms = np.sqrt(np.sum(xlemoo_obj**2, axis=1))

print(f"{'':>20} {'NSGA-II':>10} {'XLEMOO-NSGA2':>14}")
print(f"{'Solutions found':>20} {len(nsga2_obj):>10} {len(xlemoo_obj):>14}")
print(f"{'Median norm':>20} {np.median(nsga2_norms):>10.4f} {np.median(xlemoo_norms):>14.4f}")
print(f"{'Mean norm':>20} {np.mean(nsga2_norms):>10.4f} {np.mean(xlemoo_norms):>14.4f}")

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Scatter3d(
        x=nsga2_obj[:, 0],
        y=nsga2_obj[:, 1],
        z=nsga2_obj[:, 2],
        mode="markers",
        marker=dict(size=3, color="blue", opacity=0.6),
        name="NSGA-II",
    )
)

fig.add_trace(
    go.Scatter3d(
        x=xlemoo_obj[:, 0],
        y=xlemoo_obj[:, 1],
        z=xlemoo_obj[:, 2],
        mode="markers",
        marker=dict(size=3, color="red", opacity=0.6),
        name="XLEMOO-NSGA2",
    )
)

fig.update_layout(
    title="NSGA-II vs XLEMOO-NSGA2 on DTLZ2",
    scene=dict(xaxis_title="f_1", yaxis_title="f_2", zaxis_title="f_3"),
)
fig.show(renderer="notebook", include_plotlyjs="cdn")

## Summary

XLEMOO extends standard evolutionary algorithms with interpretable machine
learning to provide both optimization and explainability. Key takeaways:

- Use `xlemoo_nsga2_options()` for a ready-to-run configuration.
- The `learning_selection` field in `Template3Options` controls all ML
  parameters.
- Use the **ASF** fitness indicator with a decision-maker-supplied reference
  point to focus the learning mode on the preferred region of the Pareto
  front.
- Tune `darwin_iterations_per_cycle` and `learning_iterations_per_cycle`
  to balance exploration vs. exploitation.
- Use the `learning_archive` and rule extraction utilities for post-hoc
  analysis and explainability.